# WordCount

## Load data

In [ ]:
lines = sc.textFile('datasets/slurmd/slurmd.log.c6601')

In [4]:
lines.takeSample(withReplacement=False, num = 5) #Mostramos 5 aleatorias

[u'1494920290 2017 May 16 09:38:10 c6601 user info slurmstepd in _window_manager',
 u'1484749103 2017 Jan 18 15:18:23 c6601 user info slurmstepd task/cgroup: /slurm/uid_12244/job_414569/step_0: alloc=5120MB mem.limit=5120MB memsw.limit=unlimited',
 u'1484579003 2017 Jan 16 16:03:23 c6601 daemon info slurmd launch task 412592.0 request from 12249.113@10.118.68.11 (port 30688)',
 u'1487760351 2017 Feb 22 11:45:51 c6601 user info slurmstepd task/cgroup: /slurm/uid_12525/job_454738: alloc=5120MB mem.limit=5120MB memsw.limit=unlimited',
 u'1490784682 2017 Mar 29 12:51:22 c6601 daemon info slurmd launch task 578020.0 request from 12099.104@10.118.68.12 (port 1472)']

## Split lines into words

In [23]:
words = lines.flatMap(lambda w: w.split(' ')) # separamos las lineas en palabras (cada palabra está separada por espacios por eso (' '))
# flatMap desglosa mejor la información

In [26]:
words.take(10) # vemos que nos la separa

[u'1482336831',
 u'2016',
 u'Dec',
 u'21',
 u'17:13:51',
 u'c6601',
 u'daemon',
 u'info',
 u'slurmd',
 u'launch']

In [21]:
words = lines.map(lambda w: w.split(' '))
words.take(5) 
# vemos que map te hace una lista de lista, lo que en algunos casos no es util

[[u'1482336831',
  u'2016',
  u'Dec',
  u'21',
  u'17:13:51',
  u'c6601',
  u'daemon',
  u'info',
  u'slurmd',
  u'launch',
  u'task',
  u'387796.0',
  u'request',
  u'from',
  u'10946.113@10.118.68.11',
  u'(port',
  u'42900)'],
 [u'1482336831',
  u'2016',
  u'Dec',
  u'21',
  u'17:13:51',
  u'c6601',
  u'daemon',
  u'err',
  u'slurmd',
  u'error:',
  u'gres/mic',
  u'unable',
  u'to',
  u'set',
  u'OFFLOAD_DEVICES,',
  u'no',
  u'device',
  u'files',
  u'configured'],
 [u'1482336831',
  u'2016',
  u'Dec',
  u'21',
  u'17:13:51',
  u'c6601',
  u'daemon',
  u'info',
  u'slurmd',
  u'_run_prolog:',
  u'run',
  u'job',
  u'script',
  u'took',
  u'usec=60615'],
 [u'1482336831',
  u'2016',
  u'Dec',
  u'21',
  u'17:13:51',
  u'c6601',
  u'daemon',
  u'info',
  u'slurmd',
  u'_run_prolog:',
  u'prolog',
  u'with',
  u'lock',
  u'for',
  u'job',
  u'387796',
  u'ran',
  u'for',
  u'0',
  u'seconds'],
 [u'1482336899',
  u'2016',
  u'Dec',
  u'21',
  u'17:14:59',
  u'c6601',
  u'daemon',
  u'i

## Transform in a Pair RDD: word -> (word, 1)

In [30]:
counts = words.map(lambda x: (x,1)) # Le agrega el numero 1 a cada palabra
counts.take(10)

[(u'1482336831', 1),
 (u'2016', 1),
 (u'Dec', 1),
 (u'21', 1),
 (u'17:13:51', 1),
 (u'c6601', 1),
 (u'daemon', 1),
 (u'info', 1),
 (u'slurmd', 1),
 (u'launch', 1)]

## Aggregate counts

In [40]:
aggregated = counts.map(lambda x: x).reduceByKey(lambda x,y: x + y) # counts.map(lambda x: x) nos agarra la palabra
#reduceByKey(lambda x,y: x + y) -> nos agarra la palabra y la clave y suma la cantidad de veces que se repite esa palabra

aggregated.take(10)

[(u'', 8073),
 (u'11:59:51', 8),
 (u'3986)', 1),
 (u'11:59:55', 8),
 (u'47233)', 1),
 (u'/slurm/uid_10795/job_392998/step_0:', 1),
 (u'37092)', 1),
 (u'usec=34154', 1),
 (u'19:10:03', 1),
 (u'/slurm/uid_11953/job_416890:', 1)]

## Show the 10 most common words

Invert the tuple contents so that the key is the number of occurrences

In [58]:
result = aggregated.map(lambda (x,y): (y,x)) # cambiamos el lugar (la clave pasa a ir primero antes de la palabra)
result.sortByKey(False).take(10) # hacemos un sort por la clave (False para que vaya de mayor a menor ) y que nos agarre 10 

[(29478, u'c6601'),
 (27084, u'2017'),
 (19924, u'info'),
 (15476, u'slurmstepd'),
 (15446, u'user'),
 (12439, u'slurmd'),
 (12384, u'daemon'),
 (9285, u'job'),
 (8073, u''),
 (7911, u'err')]

Sort and take the 10 first elements:

[(8073, u''),
 (8, u'11:59:51'),
 (1, u'531335'),
 (1, u'750633'),
 (1, u'47233)'),
 (1, u'11:55:35'),
 (1, u'37092)'),
 (1, u'usec=34154'),
 (1, u'19:10:03'),
 (1, u'/slurm/uid_11953/job_416890:')]